In [2]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Set up the game window
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
window = pygame.display.set_mode((WINDOW_WIDTH + 250, WINDOW_HEIGHT))
pygame.display.set_caption("Plants vs. Zombies")

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

# Set up the game board
BOARD_ROWS = 5
BOARD_COLS = 9
CELL_SIZE = 80


# Set up the UI
UI_WIDTH = 300
UI_HEIGHT = WINDOW_HEIGHT
UI_BG_COLOR = (200, 200, 200)
UI_FONT = pygame.font.Font(None, 24)

# Load and scale plant images
PLANT_SIZE = (60, 60)
peashooter_image = pygame.image.load("peashooter.png")
peashooter_image = pygame.transform.scale(peashooter_image, PLANT_SIZE)
sunflower_image = pygame.image.load("sunflower.png")
sunflower_image = pygame.transform.scale(sunflower_image, PLANT_SIZE)
# Load zombie image
zombie_image = pygame.image.load("zombie.png")
zombie_image = pygame.transform.scale(zombie_image, (CELL_SIZE, CELL_SIZE))

class Plant:
    def __init__(self, x, y, health, damage, image):
        self.x = x
        self.y = y
        self.health = health
        self.damage = damage
        self.image = image

    def draw(self, window):
        window.blit(self.image, (self.x + (CELL_SIZE - PLANT_SIZE[0]) // 2, self.y + (CELL_SIZE - PLANT_SIZE[1]) // 2))

        
    def update(self):
        pass

    def take_damage(self, amount):
        self.health -= amount
        if self.health <= 0:
            self.die()

    def die(self):
        plants.remove(self)

# Peashooter class
# Peashooter class
class Peashooter(Plant):
    def __init__(self, x, y):
        super().__init__(x, y, 100, 25, peashooter_image)
        self.shoot_cooldown = 0

    def update(self):
        self.shoot_cooldown -= 1
        if self.shoot_cooldown <= 0:
            projectile = Projectile(self.x + CELL_SIZE, self.y + CELL_SIZE // 2, self.damage, 5)
            projectiles.append(projectile)
            self.shoot_cooldown = 60

# Sunflower class
class Sunflower(Plant):
    def __init__(self, x, y):
        super().__init__(x, y, 50, 0, sunflower_image)
        self.sun_cooldown = 0

    def update(self):
        self.sun_cooldown -= 1
        if self.sun_cooldown <= 0:
            global sun_resource
            sun_resource += 25
            self.sun_cooldown = 120

# Zombie class
class Zombie:
    def __init__(self, x, y, health, speed, image):
        self.x = x
        self.y = y
        self.health = health
        self.speed = speed
        self.image = image

    def draw(self, window):
        window.blit(self.image, (self.x, self.y))

    def move(self):
        self.x -= self.speed

    def take_damage(self, amount):
        self.health -= amount
        if self.health <= 0:
            self.die()

    def die(self):
        zombies.remove(self)

# Normal Zombie class
class NormalZombie(Zombie):
    def __init__(self, x, y):
        super().__init__(x, y, 100, 1, zombie_image)

# Conehead Zombie class
class ConeheadZombie(Zombie):
    def __init__(self, x, y):
        super().__init__(x, y, 200, 0.5, zombie_image)

# Projectile class
class Projectile:
    def __init__(self, x, y, damage, speed):
        self.x = x
        self.y = y
        self.damage = damage
        self.speed = speed

    def draw(self, window):
        pygame.draw.circle(window, YELLOW, (self.x, self.y), 5)

    def move(self):
        self.x += self.speed

    def collide(self, zombie):
        if (
            self.x >= zombie.x and
            self.x <= zombie.x + CELL_SIZE and
            self.y >= zombie.y and
            self.y <= zombie.y + CELL_SIZE
        ):
            zombie.take_damage(self.damage)
            projectiles.remove(self)
            return True
        return False
    
# Level class
class Level:
    def __init__(self, level_number, zombie_count, zombie_health, zombie_speed):
        self.level_number = level_number
        self.zombie_count = zombie_count
        self.zombie_health = zombie_health
        self.zombie_speed = zombie_speed
        
# Define levels
levels = [
    Level(1, 5, 100, 1),
    Level(2, 8, 150, 1.5),
    Level(3, 12, 200, 2),
    # Add more levels as needed
]

selected_plant = None
level_index = 0
current_level = levels[level_index]
zombies_spawned = 0
game_over = False
game_completed = False


# Create lists to store game objects
plants = []
zombies = []
projectiles = []

# Set up the game resources
sun_resource = 100

# Set up the game clock
clock = pygame.time.Clock()

# Draw UI
def draw_ui(window):
    ui_rect = pygame.Rect(WINDOW_WIDTH, 0, UI_WIDTH, UI_HEIGHT)
    pygame.draw.rect(window, UI_BG_COLOR, ui_rect)

    # Display sun resource
    sun_text = UI_FONT.render(f"Sun: {sun_resource}", True, BLACK)
    window.blit(sun_text, (WINDOW_WIDTH + 20, 20))

    # Display available plants
    plant_text = UI_FONT.render("Available Plants:", True, BLACK)
    window.blit(plant_text, (WINDOW_WIDTH + 20, 60))

    # Peashooter
    peashooter_rect = peashooter_image.get_rect()
    peashooter_rect.topleft = (WINDOW_WIDTH + 20, 100)
    peashooter_scaled = pygame.transform.scale(peashooter_image, (50, 50))
    window.blit(peashooter_scaled, peashooter_rect)
    peashooter_text = UI_FONT.render("Peashooter (50 Sun)", True, BLACK)
    window.blit(peashooter_text, (WINDOW_WIDTH + 80, 110))

    # Sunflower
    sunflower_rect = sunflower_image.get_rect()
    sunflower_rect.topleft = (WINDOW_WIDTH + 20, 160)
    sunflower_scaled = pygame.transform.scale(sunflower_image, (50, 50))
    window.blit(sunflower_scaled, sunflower_rect)
    sunflower_text = UI_FONT.render("Sunflower (25 Sun)", True, BLACK)
    window.blit(sunflower_text, (WINDOW_WIDTH + 80, 170))

    return peashooter_rect, sunflower_rect

def draw_game_over(window):
    game_over_text = UI_FONT.render("Game Over", True, RED)
    game_over_rect = game_over_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
    window.blit(game_over_text, game_over_rect)

    restart_text = UI_FONT.render("Press R to Restart", True, BLACK)
    restart_rect = restart_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 50))
    window.blit(restart_text, restart_rect)

# Game loop
selected_plant = None

running = True
while running:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over:
            # Get the mouse click position
            x, y = pygame.mouse.get_pos()
            col = x // CELL_SIZE
            row = y // CELL_SIZE
            if 0 <= col < BOARD_COLS and 0 <= row < BOARD_ROWS:
                if selected_plant == "Peashooter" and sun_resource >= 50:
                    plant = Peashooter(col * CELL_SIZE, row * CELL_SIZE)
                    plants.append(plant)
                    sun_resource -= 50
                elif selected_plant == "Sunflower" and sun_resource >= 25:
                    plant = Sunflower(col * CELL_SIZE, row * CELL_SIZE)
                    plants.append(plant)
                    sun_resource -= 25
            else:
                # Check if the click is on the UI
                peashooter_rect, sunflower_rect = draw_ui(window)
                if peashooter_rect.collidepoint(event.pos):
                    selected_plant = "Peashooter"
                elif sunflower_rect.collidepoint(event.pos):
                    selected_plant = "Sunflower"
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_r and game_over:
            # Restart the game
            level_index = 0
            current_level = levels[level_index]
            zombies_spawned = 0
            game_over = False
            plants.clear()
            zombies.clear()
            projectiles.clear()
    
    if zombies_spawned < current_level.zombie_count and not game_over and not game_completed:
        if random.randint(1, 200) == 1:  # Adjust the probability of spawning zombies
            zombie_type = random.choice([NormalZombie, ConeheadZombie])
            zombie = zombie_type(WINDOW_WIDTH, random.randint(0, BOARD_ROWS - 1) * CELL_SIZE)
            zombie.health = current_level.zombie_health
            zombie.speed = current_level.zombie_speed
            zombies.append(zombie)
            zombies_spawned += 1

    if zombies_spawned == current_level.zombie_count and len(zombies) == 0 and not game_over and not game_completed:
        level_index += 1
        if level_index < len(levels):
            current_level = levels[level_index]
            zombies_spawned = 0
            plants.clear()
            zombies.clear()
            projectiles.clear()
            sun_resource = 100
        else:
            # All levels completed
            game_completed = True
        print(level_index, game_completed)
        

    # Update game objects
    for plant in plants:
        plant.update()

    for zombie in zombies:
        zombie.move()
        if zombie.x < 0:
            game_over = True
            break
            

    for projectile in projectiles:
        projectile.move()
        for zombie in zombies:
            if projectile.collide(zombie):
                break

    # Clear the window
    window.fill(BLACK)

    # Draw game objects
    for plant in plants:
        plant.draw(window)
    for zombie in zombies:
        zombie.draw(window)
    for projectile in projectiles:
        projectile.draw(window)

    # Draw the game board
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            cell_rect = pygame.Rect(col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(window, WHITE, cell_rect, 1)

    # Draw UI
    if game_over:
        draw_game_over(window)
    elif game_completed:
        # Display victory message
        victory_text = UI_FONT.render("Congratulations! You Win!", True, GREEN)
        victory_rect = victory_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
        window.blit(victory_text, victory_rect)

        restart_text = UI_FONT.render("Press R to Restart", True, BLACK)
        restart_rect = restart_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 50))
        window.blit(restart_text, restart_rect)
    else:
        draw_ui(window)

    # Update the display
    pygame.display.update()

    # Set the game clock
    clock.tick(60)

# Quit the game
pygame.quit()

1 False
2 False


KeyboardInterrupt: 